# CV 문서분류 경진대회 - ConvNeXt Base 384 (간결화 버전)
## 5-Fold Cross Validation + Ensemble TTA

**성능 목표**: CV F1 0.95+ 유지

## 1. 환경 설정 및 라이브러리

In [1]:
import os
import time
import random
import copy

import timm
import torch
import cv2
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 시드 고정
SEED = 42
def seed_everything(seed: int = SEED):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # CUDNN 결정성
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def _seed_worker(worker_id: int):
    # 각 DataLoader worker에 고유 seed 부여
    worker_seed = SEED + worker_id
    random.seed(worker_seed)
    np.random.seed(worker_seed)
    torch.manual_seed(worker_seed)

g = torch.Generator()
g.manual_seed(SEED)
seed_everything(SEED)

## 2. 설정 및 데이터셋

In [3]:
# 설정 통합
CONFIG = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'data_path': '../data/',
    'model_name': 'convnext_base_384_in22ft1k',
    'img_size': 384,
    'lr': 2e-4,
    'epochs': 20,
    'batch_size': 48,
    'num_workers': 32,
    'n_folds': 5,
    'label_smoothing': 0.05
}

print(f"Device: {CONFIG['device']}")
print(f"Model: {CONFIG['model_name']}")

Device: cuda
Model: convnext_base_384_in22ft1k


In [4]:
def mixup_data(x, y, alpha=1.0, device=None):
    """Mixup 데이터 증강 - 디바이스 하드코딩 제거"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    # 디바이스 하드코딩 제거
    if device is None:
        device = x.device
    index = torch.randperm(batch_size, device=device)
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

class ImageDataset(Dataset):
    """적응형 Hard Augmentation + 검증 전용 Transform 분리"""
    def __init__(self, data, path, total_epochs=10, is_train=True):
        # 컬럼명 보존을 위해 DataFrame 형태 유지
        if isinstance(data, (str, os.PathLike)):
            self.df = pd.read_csv(data)
        else:
            self.df = data.copy()
        self.df.reset_index(drop=True, inplace=True)
        
        self.path = path
        self.is_train = is_train
        self.total_epochs = int(total_epochs)
        self.current_epoch = 0
        self.p_hard = 0.2  # 초기값만 설정
        
        # 컬럼명 자동 탐지 (열 순서 바뀌어도 안전)
        self.image_col = next((c for c in ["image","img_path","image_path","file","filename","name"]
                               if c in self.df.columns), self.df.columns[0])
        self.target_col = next((c for c in ["target","label","class","y"]
                               if c in self.df.columns), self.df.columns[1])
        
        # 검증용 클린 Transform (증강 없음)
        self.val_transforms = A.Compose([
            A.LongestMaxSize(max_size=CONFIG['img_size']),
            A.PadIfNeeded(min_height=CONFIG['img_size'], min_width=CONFIG['img_size'], border_mode=0, value=0),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
        
        # 학습용만 증강 Transform 초기화
        self._update_transforms()  # 첫 배치 전 비지 않게 (검증일 땐 내부에서 no-op)

    def set_epoch(self, epoch):
        """에포크 업데이트 메서드 (학습용만) - p_hard 계산 통합"""
        if self.is_train:
            epoch = int(epoch)
            self.current_epoch = epoch
            self.p_hard = 0.2 + 0.3 * (epoch / max(1, self.total_epochs - 1))
            self.p_hard = float(min(1.0, max(0.0, self.p_hard)))  # 안전 클램프
            self._update_transforms()
            self._update_transforms()
    
    def _update_transforms(self):
        """에포크에 따른 증강 변환 업데이트 (학습용만) - p_hard 계산 제거"""
        if not self.is_train:
            self.normal_aug = None
            self.hard_aug = None
            return
        
        # p_hard는 set_epoch에서만 계산되므로 여기서는 제거
        
        # Normal augmentation
        self.normal_aug = A.Compose([
            A.LongestMaxSize(max_size=CONFIG['img_size']),
            A.PadIfNeeded(min_height=CONFIG['img_size'], min_width=CONFIG['img_size'], border_mode=0, value=0),
            A.OneOf([
                A.Rotate(limit=[90,90], p=1.0),
                A.Rotate(limit=[180,180], p=1.0),
                A.Rotate(limit=[270,270], p=1.0),
            ], p=0.6),
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8),
            A.GaussNoise(var_limit=(30.0, 100.0), p=0.7),
            A.HorizontalFlip(p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
        
        # Hard augmentation
        self.hard_aug = A.Compose([
            A.LongestMaxSize(max_size=CONFIG['img_size']),
            A.PadIfNeeded(min_height=CONFIG['img_size'], min_width=CONFIG['img_size'], border_mode=0, value=0),
            A.OneOf([
                A.Rotate(limit=[90,90], p=1.0),
                A.Rotate(limit=[180,180], p=1.0),
                A.Rotate(limit=[270,270], p=1.0),
                A.Rotate(limit=[-15,15], p=1.0),
            ], p=0.8),
            A.OneOf([
                A.MotionBlur(blur_limit=15, p=1.0),
                A.GaussianBlur(blur_limit=15, p=1.0),
            ], p=0.95),
            A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.9),
            A.GaussNoise(var_limit=(50.0, 150.0), p=0.8),
            A.JpegCompression(quality_lower=70, quality_upper=100, p=0.5),
            A.HorizontalFlip(p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # DataFrame 인덱싱으로 컬럼명 보존
        row = self.df.iloc[idx]
        name   = str(row[self.image_col])   # 이름/경로 문자열화
        target = int(row[self.target_col])  # 라벨 정수화
        img = np.array(Image.open(os.path.join(self.path, name)).convert('RGB'))
        
        # 핵심 수정: 학습/검증 분리
        if self.is_train:
            # 학습: 적응형 증강 적용
            if random.random() < self.p_hard:
                img = self.hard_aug(image=img)['image']
            else:
                img = self.normal_aug(image=img)['image']
        else:
            # 검증: 클린한 전처리만 적용
            aug = self.normal_aug or self.val_transforms  # 널가드
            img = aug(image=img)['image']
        
        return img, target

## 3. 학습 및 검증 함수

In [5]:
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    """한 에포크 학습"""
    scaler = GradScaler()
    model.train()
    total_loss, preds_list, targets_list = 0, [], []

    for image, targets in tqdm(loader, desc="Training"):
        image, targets = image.to(device), targets.to(device)
        
        # Mixup 적용 (30% 확률)
        if random.random() < 0.3:
            mixed_x, y_a, y_b, lam = mixup_data(image, targets, alpha=1.0)
            with autocast(): 
                preds = model(mixed_x)
            loss = lam * loss_fn(preds, y_a) + (1 - lam) * loss_fn(preds, y_b)
        else:
            with autocast(): 
                preds = model(image)
            loss = loss_fn(preds, targets)

        model.zero_grad(set_to_none=True)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

    return {
        "train_loss": total_loss / len(loader),
        "train_acc": accuracy_score(targets_list, preds_list),
        "train_f1": f1_score(targets_list, preds_list, average='macro'),
    }

def validate_one_epoch(loader, model, loss_fn, device):
    """한 에포크 검증"""
    model.eval()
    total_loss, n_samples = 0.0, 0
    preds_list, targets_list = [], []
    use_amp = torch.cuda.is_available()

    with torch.inference_mode():
        for images, targets in tqdm(loader, desc="Validating", leave=False):
            images = images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)

            with torch.cuda.amp.autocast(enabled=use_amp):
                logits = model(images)
                loss = loss_fn(logits, targets)

            bsz = images.size(0)
            total_loss += loss.item() * bsz
            n_samples += bsz

            preds_list.extend(logits.argmax(dim=1).detach().cpu().tolist())
            targets_list.extend(targets.detach().cpu().tolist())

    avg_loss = total_loss / max(1, n_samples)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1  = f1_score(targets_list, preds_list, average="macro")

    return {"val_loss": avg_loss, "val_acc": val_acc, "val_f1": val_f1}

## 4. K-Fold Cross Validation

In [6]:
# train_single_fold 함수 수정 - 데이터셋 epoch 업데이트 추가
def train_single_fold(fold, train_idx, val_idx, train_df):
    """단일 Fold 학습"""
    print(f"\n{'='*50}\nFOLD {fold + 1}/{CONFIG['n_folds']}\n{'='*50}")
    
    # 데이터 분할
    train_fold_df = train_df.iloc[train_idx].reset_index(drop=True)
    val_fold_df = train_df.iloc[val_idx].reset_index(drop=True)
    
    # 데이터셋 및 로더 생성
    trn_dataset = ImageDataset(train_fold_df, CONFIG['data_path'] + "train/", 
                              total_epochs=CONFIG['epochs'], is_train=True)
    val_dataset = ImageDataset(val_fold_df, CONFIG['data_path'] + "train/", 
                              total_epochs=CONFIG['epochs'], is_train=False)
    
    trn_loader = DataLoader(trn_dataset, batch_size=CONFIG['batch_size'], shuffle=True, 
                           num_workers=CONFIG['num_workers'], pin_memory=True, drop_last=False,
                           persistent_workers=True, worker_init_fn=_seed_worker, generator=g)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, 
                           num_workers=CONFIG['num_workers'], pin_memory=True, persistent_workers=True,
                           worker_init_fn=_seed_worker, generator=g)
    
    print(f"Train samples: {len(trn_dataset)}, Validation samples: {len(val_dataset)}")
    
    # 모델 및 최적화 설정
    model = timm.create_model(CONFIG['model_name'], pretrained=True, num_classes=17).to(CONFIG['device'])
    loss_fn = nn.CrossEntropyLoss(label_smoothing=CONFIG['label_smoothing'])
    optimizer = Adam(model.parameters(), lr=CONFIG['lr'])
    scheduler = CosineAnnealingLR(optimizer, T_max=CONFIG['epochs'])
    
    best_val_f1 = 0.0
    best_model = None
    
    # 학습 루프
    for epoch in range(CONFIG['epochs']):
        # ★ 핵심: 매 에포크마다 데이터셋 업데이트
        trn_dataset.set_epoch(epoch)
        
        train_ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, CONFIG['device'])
        val_ret = validate_one_epoch(val_loader, model, loss_fn, CONFIG['device'])
        scheduler.step()
        
        print(f"Epoch {epoch+1:2d} | Train Loss: {train_ret['train_loss']:.4f} | "
              f"Train F1: {train_ret['train_f1']:.4f} | Val Loss: {val_ret['val_loss']:.4f} | "
              f"Val F1: {val_ret['val_f1']:.4f}")
        
        if val_ret['val_f1'] > best_val_f1:
            best_val_f1 = val_ret['val_f1']
            best_model = copy.deepcopy(model.state_dict())
    
    # GPU 메모리 정리
    torch.cuda.empty_cache()
    
    print(f"Fold {fold + 1} Best Validation F1: {best_val_f1:.4f}")
    return best_val_f1, best_model

# K-Fold 실행
train_df = pd.read_csv(CONFIG['data_path'] + "train.csv")
skf = StratifiedKFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=SEED)

fold_results = []
fold_models = []

print(f"Starting {CONFIG['n_folds']}-Fold Cross Validation...")

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    best_f1, best_model = train_single_fold(fold, train_idx, val_idx, train_df)
    fold_results.append(best_f1)
    fold_models.append(best_model)

# 결과 요약
mean_f1, std_f1 = np.mean(fold_results), np.std(fold_results)
print(f"\n{'='*60}\nK-FOLD CROSS VALIDATION RESULTS\n{'='*60}")
for i, f1 in enumerate(fold_results):
    print(f"Fold {i+1}: {f1:.4f}")
print(f"\nMean CV F1: {mean_f1:.4f} ± {std_f1:.4f}")
print(f"Best single fold: {max(fold_results):.4f}")

Starting 5-Fold Cross Validation...

FOLD 1/5
Train samples: 1256, Validation samples: 314


Training: 100%|██████████| 27/27 [00:22<00:00,  1.21it/s]


Epoch  1 | Train Loss: 2.2499 | Train F1: 0.3452 | Val Loss: 1.2059 | Val F1: 0.6476


Training: 100%|██████████| 27/27 [00:16<00:00,  1.64it/s]


Epoch  2 | Train Loss: 1.3915 | Train F1: 0.5154 | Val Loss: 0.8083 | Val F1: 0.7822


...

Epoch 18 | Train Loss: 0.6074 | Train F1: 0.8168 | Val Loss: 0.5043 | Val F1: 0.9087


Training: 100%|██████████| 27/27 [00:17<00:00,  1.56it/s]


Epoch 19 | Train Loss: 0.5719 | Train F1: 0.8569 | Val Loss: 0.5090 | Val F1: 0.9113


Training: 100%|██████████| 27/27 [00:16<00:00,  1.62it/s]


Epoch 20 | Train Loss: 0.7087 | Train F1: 0.8363 | Val Loss: 0.5071 | Val F1: 0.9079
Fold 5 Best Validation F1: 0.9244

K-FOLD CROSS VALIDATION RESULTS
Fold 1: 0.9507
Fold 2: 0.9524
Fold 3: 0.9326
Fold 4: 0.9247
Fold 5: 0.9244

Mean CV F1: 0.9369 ± 0.0123
Best single fold: 0.9524


In [7]:
# 클래스별 성능 시각화
def classwise_accuracy(model, loader, device, num_classes=None):
    model.eval()
    use_amp = torch.cuda.is_available()
    C = None
    correct = None
    counts  = None

    with torch.inference_mode():
        for images, targets in loader:
            images  = images.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)

            with torch.cuda.amp.autocast(enabled=use_amp):
                logits = model(images)
            preds = logits.argmax(1)

            if C is None:
                C = num_classes if num_classes is not None else logits.shape[1]
                correct = torch.zeros(C, dtype=torch.long, device=targets.device)
                counts  = torch.zeros(C, dtype=torch.long, device=targets.device)

            # 클래스별 개수 / 정답 개수 집계 (벡터화)
            counts  += torch.bincount(targets, minlength=C)
            correct += torch.bincount(targets[preds == targets], minlength=C)

    acc = (correct.float() / counts.clamp(min=1).float()).cpu().numpy()
    return acc, counts.cpu().numpy()

# 평가 대상: 현재 검증 로더와 모델 사용
num_classes = CONFIG.get("num_classes", 17)
acc, counts = classwise_accuracy(model, val_loader, CONFIG["device"], num_classes=num_classes)

# 라벨 (원하는 이름이 있으면 여기서 교체: 예) CLASS_NAMES)
class_names = [f"C{i}" for i in range(len(acc))]

# --- Plot ---
plt.figure(figsize=(16, 6))
bars = plt.bar(range(len(acc)), acc * 100)
plt.xticks(range(len(acc)), class_names)
plt.ylim(0, 105)
plt.ylabel("Accuracy (%)")
plt.title("Class-wise Prediction Accuracy")

# 막대 위에 퍼센트 표시
for b, a in zip(bars, acc):
    h = (a * 100)
    plt.text(b.get_x() + b.get_width()/2, h + 1, f"{h:.1f}%", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

In [8]:
# # ==== FULL/샘플 스캔로 EDA (취약클래스 보강용) ====

# # 0) 경로/칼럼명 정합
# IMG_COL = img  # 너의 DF 칼럼명에 맞게: 'img_path' 등으로 교체
# TARGET_COL = "target"

# # 1) 스캔 (전수 권장. 느리면 SAMPLE_N=5000 등으로 제한)
# SAMPLE_N = None  # None이면 전수, 정수면 샘플 수
# df_scan = train_df.sample(SAMPLE_N, random_state=42) if SAMPLE_N else train_df

# stats = []
# for p, y in tqdm(zip(df_scan[IMG_COL].values, df_scan[TARGET_COL].values), total=len(df_scan), desc="Scanning"):
#     img = cv2.imdecode(np.fromfile(p, dtype=np.uint8), cv2.IMREAD_COLOR)  # 한글 경로 호환
#     if img is None: 
#         continue
#     h, w = img.shape[:2]
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#     # 밝기(평균), 대비(표준편차), 블러(라플라시안 분산) — 간단하지만 강력
#     bright = float(gray.mean())
#     contrast = float(gray.std())
#     blur = float(cv2.Laplacian(gray, cv2.CV_64F).var())

#     stats.append((p, y, w, h, w/h, bright, contrast, blur))

# eda_df = pd.DataFrame(stats, columns=[IMG_COL, TARGET_COL, "w", "h", "ar", "bright", "contrast", "blur"])

# # 2) 클래스 분포 + 품질지표 요약
# class_counts = eda_df[TARGET_COL].value_counts().sort_index()
# per_class = eda_df.groupby(TARGET_COL)[["w","h","ar","bright","contrast","blur"]].agg(["median","mean","std"]).round(2)

# print("클래스 분포(개수):")
# display(class_counts.to_frame("count"))

# print("\n클래스별 화질/밝기/블러 통계:")
# display(per_class)

# # 3) 취약 후보 탐색 규칙 (예시: 블러 중앙값이 낮거나 밝기가 낮은 클래스 TOP k)
# weak_by_blur = eda_df.groupby(TARGET_COL)["blur"].median().sort_values().head(5)
# weak_by_dark = eda_df.groupby(TARGET_COL)["bright"].median().sort_values().head(5)

# print("\n블러 취약(중앙값 낮은 순) TOP5:")
# display(weak_by_blur)

# print("\n저조도 취약(밝기 중앙값 낮은 순) TOP5:")
# display(weak_by_dark)


In [9]:
# # ==== Confusion Matrix & Hard Samples ====
# model.eval()
# all_preds, all_targets, all_paths = [], [], []

# with torch.inference_mode(), torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
#     for (images, targets, *maybe_paths) in tqdm(val_loader, desc="Eval/CM"):
#         images = images.to(CONFIG["device"], non_blocking=True)
#         logits = model(images)
#         preds = logits.argmax(1).detach().cpu().numpy()
#         all_preds.extend(preds)
#         all_targets.extend(targets.numpy())
#         # Dataset에서 이미지 경로를 반환하지 않는다면, 아래는 생략
#         if maybe_paths:
#             all_paths.extend(maybe_paths[0])

# all_preds = np.array(all_preds)
# all_targets = np.array(all_targets)
# labels = np.arange(CONFIG.get("num_classes", all_preds.max()+1))

# # 1) 혼동행렬
# cm = confusion_matrix(all_targets, all_preds, labels=labels, normalize=None)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[f"C{i}" for i in labels])
# fig, ax = plt.subplots(figsize=(8, 8))
# disp.plot(ax=ax, cmap="Blues", colorbar=False, xticks_rotation=45)
# plt.title("Confusion Matrix (counts)")
# plt.tight_layout()
# plt.show()

# print("\nClassification Report (macro avg 확인):")
# print(classification_report(all_targets, all_preds, digits=4))

# # 2) 클래스별 최악 샘플 저장 (정답 y, 예측 yhat가 다른 케이스)
# os.makedirs("hard_samples", exist_ok=True)
# TOP_K = 24  # 클래스별 저장 개수
# if len(all_paths) == len(all_targets):  # 경로가 있는 경우에만
#     for c in labels:
#         bad_idx = np.where((all_targets == c) & (all_preds != c))[0][:TOP_K]
#         grid = []
#         for i in bad_idx:
#             p = all_paths[i]
#             img = cv2.imdecode(np.fromfile(p, dtype=np.uint8), cv2.IMREAD_COLOR)
#             if img is None: 
#                 continue
#             img = cv2.resize(img, (224, 224))
#             grid.append(img)
#         if grid:
#             rows = int(np.ceil(len(grid)/6))
#             canvas = np.zeros((rows*224, 6*224, 3), dtype=np.uint8)
#             for j, im in enumerate(grid):
#                 r, c2 = divmod(j, 6)
#                 canvas[r*224:(r+1)*224, c2*224:(c2+1)*224] = im
#             cv2.imwrite(os.path.join("hard_samples", f"class_{c}_hard.jpg"), canvas)


## 5. TTA 추론 및 앙상블

In [10]:
# TTA 변형 정의
tta_transforms = [
    # 원본
    A.Compose([
        A.LongestMaxSize(max_size=CONFIG['img_size']),
        A.PadIfNeeded(min_height=CONFIG['img_size'], min_width=CONFIG['img_size'], border_mode=0, value=0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    # 90도 회전들
    A.Compose([
        A.LongestMaxSize(max_size=CONFIG['img_size']),
        A.PadIfNeeded(min_height=CONFIG['img_size'], min_width=CONFIG['img_size'], border_mode=0, value=0),
        A.Rotate(limit=[90, 90], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.LongestMaxSize(max_size=CONFIG['img_size']),
        A.PadIfNeeded(min_height=CONFIG['img_size'], min_width=CONFIG['img_size'], border_mode=0, value=0),
        A.Rotate(limit=[180, 180], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.LongestMaxSize(max_size=CONFIG['img_size']),
        A.PadIfNeeded(min_height=CONFIG['img_size'], min_width=CONFIG['img_size'], border_mode=0, value=0),
        A.Rotate(limit=[-90, -90], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    # 밝기 개선
    A.Compose([
        A.LongestMaxSize(max_size=CONFIG['img_size']),
        A.PadIfNeeded(min_height=CONFIG['img_size'], min_width=CONFIG['img_size'], border_mode=0, value=0),
        A.RandomBrightnessContrast(brightness_limit=[0.3, 0.3], contrast_limit=[0.3, 0.3], p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
]

In [11]:
class TTADataset(Dataset):
    """TTA 추론용 데이터셋"""
    def __init__(self, data, path, transforms):
        self.df = pd.read_csv(data).values if isinstance(data, str) else data.values
        self.path = path
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert("RGB"))
        
        augmented_images = []
        for transform in self.transforms:
            aug_img = transform(image=img)['image']
            augmented_images.append(aug_img)
        
        return augmented_images, target

def ensemble_tta_inference(models, loader):
    """5-Fold 모델 앙상블 + TTA 추론"""
    all_predictions = []
    
    for batch_idx, (images_list, _) in enumerate(tqdm(loader, desc="Ensemble TTA")):
        batch_size = images_list[0].size(0)
        ensemble_probs = torch.zeros(batch_size, 17).to(CONFIG['device'])
        
        # 각 fold 모델별 예측
        for model in models:
            with torch.no_grad():
                # 각 TTA 변형별 예측
                for images in images_list:
                    images = images.to(CONFIG['device'])
                    preds = model(images)
                    probs = torch.softmax(preds, dim=1)
                    ensemble_probs += probs / (len(models) * len(images_list))
        
        final_preds = torch.argmax(ensemble_probs, dim=1)
        all_predictions.extend(final_preds.cpu().numpy())
    
    return all_predictions

In [12]:
# 앙상블 모델 준비
ensemble_models = []
for state_dict in fold_models:
    model = timm.create_model(CONFIG['model_name'], pretrained=False, num_classes=17).to(CONFIG['device'])
    model.load_state_dict(state_dict)
    model.eval()
    ensemble_models.append(model)

# TTA 데이터셋 및 로더 생성
tta_dataset = TTADataset(CONFIG['data_path'] + "sample_submission.csv", 
                        CONFIG['data_path'] + "test/", tta_transforms)
tta_loader = DataLoader(tta_dataset, batch_size=64, shuffle=False, 
                       num_workers=8, pin_memory=True, persistent_workers=True,
                       worker_init_fn=_seed_worker, generator=g)

print(f"Using ensemble of {len(ensemble_models)} fold models for inference")
print(f"TTA Dataset size: {len(tta_dataset)}")

Using ensemble of 5 fold models for inference
TTA Dataset size: 3140


In [13]:
# TTA 추론 실행
print("Starting Ensemble TTA inference...")
start_time = time.time()
tta_predictions = ensemble_tta_inference(ensemble_models, tta_loader)
inference_time = time.time() - start_time

print(f"Inference completed in {inference_time//60:.0f}m {inference_time%60:.0f}s")

Starting Ensemble TTA inference...


Ensemble TTA: 100%|██████████| 50/50 [09:19<00:00, 11.19s/it]

Inference completed in 9m 20s


## 6. 결과 저장 및 검증

In [14]:
# 결과 저장
tta_pred_df = pd.DataFrame(tta_dataset.df, columns=['ID', 'target'])
tta_pred_df['target'] = tta_predictions

In [15]:
# 검증
sample_submission_df = pd.read_csv(CONFIG['data_path'] + "sample_submission.csv")
try:
    assert (sample_submission_df['ID'] == tta_pred_df['ID']).all()
    print("✓ Submission format verified")
except AssertionError:
    print("✗ Submission format error")
    raise

✓ Submission format verified


In [16]:
# 최종 저장
tta_pred_df.to_csv("../submission/choice.csv", index=False)
print("\n✓ Final predictions saved to choice.csv")
print(f"✓ CV Performance: {mean_f1:.4f} ± {std_f1:.4f}")
print(f"✓ Inference Time: {inference_time//60:.0f}m {inference_time%60:.0f}s")


✓ Final predictions saved to choice.csv
✓ CV Performance: 0.9369 ± 0.0123
✓ Inference Time: 9m 20s


In [17]:
# 샘플 출력
print("\nPrediction sample:")
tta_pred_df.head()


Prediction sample:


,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
